In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
data_path = r'C:\Users\cheng\Desktop\20240201\Question\Wimbledon_featured_matches.csv'
data = pd.read_csv(data_path)
print(data.head())

              match_id         player1        player2 elapsed_time  set_no  \
0  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   

   game_no  point_no  p1_sets  p2_sets  p1_games  ...  p2_break_pt_won  \
0        1         1        0        0         0  ...                0   
1        1         2        0        0         0  ...                0   
2        1         3        0        0         0  ...                0   
3        1         4        0        0         0  ...                0   
4        1         5        0        0         0  ...                0   

  p1_break_pt_missed p2_break_pt_missed  p1_distance_run  p2_distance_run  \
0        

In [10]:
# 按match_id分组
grouped = data.groupby('match_id')
print(grouped.size())

match_id
2023-wimbledon-1301    300
2023-wimbledon-1302    201
2023-wimbledon-1303    134
2023-wimbledon-1304    337
2023-wimbledon-1305    246
2023-wimbledon-1306    332
2023-wimbledon-1307    232
2023-wimbledon-1308    190
2023-wimbledon-1309    213
2023-wimbledon-1310    318
2023-wimbledon-1311    170
2023-wimbledon-1312    275
2023-wimbledon-1313    290
2023-wimbledon-1314    185
2023-wimbledon-1315    198
2023-wimbledon-1316    167
2023-wimbledon-1401    225
2023-wimbledon-1402    274
2023-wimbledon-1403    122
2023-wimbledon-1404    286
2023-wimbledon-1405    215
2023-wimbledon-1406    195
2023-wimbledon-1407    331
2023-wimbledon-1408    271
2023-wimbledon-1501    189
2023-wimbledon-1502    283
2023-wimbledon-1503    193
2023-wimbledon-1504    217
2023-wimbledon-1601    159
2023-wimbledon-1602    202
2023-wimbledon-1701    334
dtype: int64


In [14]:
sample_match = grouped.get_group('2023-wimbledon-1301')
print(sample_match)

                match_id         player1        player2 elapsed_time  set_no  \
0    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   
..                   ...             ...            ...          ...     ...   
295  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:53:25       4   
296  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:54:01       4   
297  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:02       4   
298  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:34       4   
299  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:56:11       4   

     game_no  point_no  p1_sets  p2_set

In [38]:
def analyse_match(match):
    match_id = match['match_id']
    player1 = match['player1']
    player2 = match['player2']
    # point_victor,winner of the point,"1 if player 1 wins, 2 if player 2 wins"
    # 统计单盘比赛中每个球员的得分情况
    set_grouped = match.groupby('set_no')
    res_dict = {'player1_score':[], 'player2_score':[], 'player1_win_double_count':[], 'player2_win_double_count':[], 'player1_loss_after_win':[], 'player2_loss_after_win':[]}
    for set_no, set_data in set_grouped:
        player1_score = 0
        player2_score = 0
        # 统计 player 连赢两盘的次数
        player1_win_double_count = 0
        player2_win_double_count = 0
        # 统计 player 连赢一盘后输掉下一盘的次数\
        player1_loss_after_win = 0
        player2_loss_after_win = 0
        flag = -1
        for index, row in set_data.iterrows():
            point_victor = row['point_victor']
            if point_victor == 1:
                player1_score += 1
                player1_win_double_count += (flag == 1)
                player2_loss_after_win += (flag == 2)
            else:
                player2_score += 1
                player2_win_double_count += (flag == 2)
                player1_loss_after_win += (flag == 1)
            flag = point_victor
        res_dict['player1_score'].append(player1_score)
        res_dict['player2_score'].append(player2_score)
        res_dict['player1_win_double_count'].append(player1_win_double_count)
        res_dict['player2_win_double_count'].append(player2_win_double_count)
        res_dict['player1_loss_after_win'].append(player1_loss_after_win)
        res_dict['player2_loss_after_win'].append(player2_loss_after_win)
    # print(res_dict)
    return res_dict
res_dict = analyse_match(sample_match)

In [39]:
# 卡方检验
def chi_square_test(res_dict):
    player1_score = res_dict['player1_score']
    player1_loss = res_dict['player2_score']
    player1_win_double_count = res_dict['player1_win_double_count']
    player1_loss_after_win = res_dict['player1_loss_after_win']
    # print(player1_score)

    # 卡方检验
    a = sum(player1_score) 
    b = sum(player1_win_double_count)
    c = sum(player1_loss)
    d = sum(player1_loss_after_win)
    chi_square_score = (a*d-b*c)**2*(a+b+c+d)/(a+b)/(c+d)/(a+c)/(b+d)
    # print(chi_square_score)
    return chi_square_score
chi_square_score = chi_square_test(res_dict)

In [41]:
# 穷举所有的比赛
chi_square_score_list = []
for match_id, match in grouped:
    res_dict = analyse_match(match)
    chi_square_score = chi_square_test(res_dict)
    chi_square_score_list.append(chi_square_score)
print(chi_square_score_list)
# 穷举所有比赛的结果求和
match_all = {'player1_score':[], 'player2_score':[], 'player1_win_double_count':[], 'player2_win_double_count':[], 'player1_loss_after_win':[], 'player2_loss_after_win':[]}
for match_id, match in grouped:
    res_dict = analyse_match(match)
    for key in match_all.keys():
        match_all[key].extend(res_dict[key])
print(match_all)
chi_square_score_all = chi_square_test(match_all)
print(chi_square_score_all)


[0.045489788006773994, 0.28398065995756905, 0.15095413743360245, 0.626537957848038, 0.018085821595030156, 0.6056275698500139, 1.5607440571727855, 0.8693503507697596, 0.005189869374980039, 1.3487253113481688, 0.9317208489752606, 0.0713344464676952, 0.001010642459877429, 0.24904396582323413, 0.5377462812620359, 0.1132486649132885, 0.13606481987458466, 0.1769526262789706, 0.15295564842090237, 0.9430987475362841, 0.3086385555079652, 1.4558592939456927, 0.7986381027690602, 3.6269255391907365, 0.5382405620079384, 0.46629861551284363, 1.7292861180531016, 2.2690570795126328e-05, 0.20895149651704195, 0.19531404579306852, 5.81841539432847e-07]
{'player1_score': [34, 44, 34, 46, 22, 34, 43, 20, 18, 13, 26, 34, 32, 29, 48, 27, 32, 34, 36, 29, 36, 36, 47, 28, 41, 36, 46, 26, 42, 22, 25, 27, 24, 39, 34, 41, 39, 23, 26, 34, 25, 8, 36, 32, 41, 38, 32, 44, 49, 30, 34, 28, 45, 38, 27, 27, 20, 13, 37, 27, 31, 35, 29, 40, 37, 37, 20, 34, 36, 19, 36, 16, 30, 36, 44, 45, 31, 21, 33, 29, 27, 39, 31, 39, 34, 